# Logistic regression

## Data Load

In [9]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tqdm import tqdm
#import fastparquet

In [10]:
from functions import get_cv_score, custom_train_test_split

In [11]:
data_extended_path = r"../data/data_extended_v1_1/data_v1_1.parquet.gzip"
df = pd.read_csv(data_extended_path)
#df = pd.read_parquet(data_extended_path, engine='fastparquet')
#pd.read_parquet(r"../data/data_extended_v1_1/data_v1_1.parquet.gzip")

In [12]:
df

,subid,firstrun.gopast,firstfix.dur,dur,total_dur,lang,FF_norm,FP_norm,TF_norm,lang_code
0,DU_04,0.0,278.0,464.0,5486.0,du,0.000000,0.050674,0.084579,0
1,DU_04,944.0,164.0,480.0,5486.0,du,0.172074,0.029894,0.087495,0
2,DU_04,155.0,155.0,155.0,5486.0,du,0.028254,0.028254,0.028254,0
3,DU_04,323.0,323.0,323.0,5486.0,du,0.058877,0.058877,0.058877,0
4,DU_04,462.0,265.0,462.0,5486.0,du,0.084214,0.048305,0.084214,0
...,...,...,...,...,...,...,...,...,...,...
1732238,tr_52,0.0,560.0,1598.0,26247.0,tr,0.000000,0.021336,0.060883,11
1732239,tr_52,0.0,432.0,1810.0,26247.0,tr,0.000000,0.016459,0.068960,11
1732240,tr_52,0.0,766.0,2109.0,26247.0,tr,0.000000,0.029184,0.080352,11
1732241,tr_52,0.0,652.0,1459.0,26247.0,tr,0.000000,0.024841,0.055587,11


## LogReg NLIR

In [13]:
logreg = LogisticRegression(C=1e5, multi_class='multinomial', solver='lbfgs', random_state=0)

In [14]:
train_cols = ['FF_norm', 'FP_norm', 'TF_norm'] # more cols for additional task of training on more columns.


In [15]:
cross_val_scores = get_cv_score(df, train_cols, "LogisticRegression", logreg)
print(np.mean(cross_val_scores))

NameError: name 'tqdm' is not defined

In [8]:
X_train, X_test, y_train, y_test, test_lang = custom_train_test_split(df, train_cols)

In [65]:
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
test_acc_score = accuracy_score(y_test, y_pred)

#print(f'Test set accuracy: {test_acc_score}; overall cross validation accuracy" {np.mean(cross_val_scores)}')

/mnt/c/Users/linas/OneDrive/Documents/ITU/Semester3/NLP/NLP_project/ANLP_project/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [53]:
# get other metrics as well, maybe store and analyze output?
# write predictions
# out path
# np.savetxt(r"../data/outputs/NLIR_predictions_logreg.csv", y_pred, delimiter=",")

### LogReg Similarity

In [66]:
X_test['lang'] = test_lang # add language column
languages = list(test_lang.unique())
dict_lang = {}
for ll in languages:
    df = X_test[X_test['lang'] == ll]
    dict_lang[ll] = df

In [ ]:
# double for loop: for each key/df in dict -> predict, record pred as dict, put to df? and get mean s meatric

In [54]:
dict_lang['du']#[:-1]

,FF_norm,FP_norm,TF_norm,lang
0,0.000000,0.050674,0.084579,du
1,0.172074,0.029894,0.087495,du
2,0.028254,0.028254,0.028254,du
3,0.058877,0.058877,0.058877,du
4,0.084214,0.048305,0.084214,du
...,...,...,...,...
1548361,0.000000,0.126645,0.152259,du
1548362,0.058281,0.058281,0.058281,du
1548363,0.136319,0.201515,0.308528,du
1548364,0.329602,0.062232,0.157392,du


In [67]:
dict_proba ={}
for key in dict_lang.keys():
    test_proba = logreg.predict_proba(dict_lang[key][train_cols])
    dict_proba[key] = test_proba
    # write logreg predicted classes as col and have key lang as rows.
    # nope lang as cols and predictions as rows bc dict structure. then can map better ?

dict_keys(['du', 'ee', 'fi', 'ge', 'gr', 'he', 'it', 'en', 'no', 'ru', 'sp', 'tr'])

In [ ]:
dict_proba ={}


In [ ]:
predictions = pd.DataFrame(dict_proba)

In [57]:
logreg.classes_ # now map from lang code to lang. write external function to map and unmap 

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [56]:
test_proba

array([[0.04921869, 0.11924527, 0.10763136, ..., 0.08447535, 0.06897935,
        0.03995224],
       [0.04515644, 0.11346616, 0.09746004, ..., 0.08097361, 0.07399716,
        0.04168536],
       [0.04814203, 0.11560505, 0.10773757, ..., 0.08345771, 0.06689759,
        0.03899432],
       ...,
       [0.06363794, 0.14859503, 0.13145295, ..., 0.09593921, 0.06338954,
        0.03767961],
       [0.04654687, 0.11844404, 0.0979013 , ..., 0.08257245, 0.07612656,
        0.04244673],
       [0.05620283, 0.13185129, 0.12173659, ..., 0.09025828, 0.0640588 ,
        0.03797565]])